# RAVDESS Facial Landmark Tracking - Dataset loading code

Sample code developed by Marco Simões (msimoes@dei.uc.pt) in the scope of the Feature Engineering course of the Bachelor in Data Science and Enginnering of Department of Informatics Engineering of the University of Coimbra, Portugal (LECD | DEI-FCTUC). All rights reserved.

In [2]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import os

datapath = 'data'
nlandmarks = 68


def preprareData(files):
    ''' Load data from csv files and save in numpy arrays '''

    # prepare auxiliar arrays
    actors = np.zeros((len(files)), dtype=np.uint8)
    emotions = np.zeros((len(files)), dtype=np.uint8)
    frames = np.zeros((len(files)), dtype=np.uint16)

    all_data = []


    for i in range(len(files)):
        # process video file
        filename = files[i]
        
        # read csv file
        dataset = pd.read_csv(datapath + '/' + filename, sep=',')
        all_data.append(np.array(dataset.loc[:, 'x_0':'y_'+str(nlandmarks-1)]))
        
        # split filename to get aux values
        parts = filename[:-4].split('-')
        actors[i] = int(parts[-1])
        emotions[i] = int(parts[2])     
        frames[i] = len(dataset.index)

    # create 3d array
    data = np.empty(shape=(len(files), nlandmarks*2, int(np.max(frames))), dtype=np.float32)
    data[:] = np.nan

    # fill the data array
    for i in range(len(files)):
        data[i, :, :int(frames[i])] = all_data[i].T

    return (data, actors, emotions, frames)



# list files in folder 
files = os.listdir(datapath)

# load data
data, actors, emotions, frames = preprareData(files)

# save data
# np.save('data.npy', data)
# np.save('actors.npy', actors)
# np.save('emotions.npy', emotions)
# np.save('frames.npy', frames)


FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [23]:
def reshapeData(data3d, emotions, actors, frames):
    ''' Convert 3d array into 2d without NaNs '''

    # prepare arrays of data, emotions and actors of shape (n, 136), (n,) and (n,) [being n the sum of all frames]
    data2d = np.empty(shape=(np.sum(frames), data.shape[1]), dtype=np.float32)
    emotions_by_frame = np.empty(shape=(np.sum(frames)), dtype=np.uint8)
    actor_by_frame = np.empty(shape=(np.sum(frames)), dtype=np.uint8)

    k = 0
    for v in range(data3d.shape[0]):
        # gets non nan values of video v
        data2d[k:k+frames[v], :] = data3d[v, :, :frames[v]].T
        
        # fills emotions and actors arrays
        emotions_by_frame[k:k+frames[v]] = emotions[v]
        actor_by_frame[k:k+frames[v]] = actors[v]
        
        # updates k 
        k += frames[v]

    return (data2d, emotions_by_frame, actor_by_frame)


def getData(actor=None, emotion=None, type='2d'):
    ''' Returns data in 2d or 3d shape. if actor is provided, filters data to return that actor. 
    if emotion is provided, filters data to return only that emotion. Cumulative filters, meaning that 
    if both are provided, the intersection of both filters is returned (only videos of that actor with that emotion).'''

    data = np.load('data.npy')
    actors = np.load('actors.npy')
    emotions = np.load('emotions.npy')
    frames = np.load('frames.npy')

    if type=='2d':
        # get 2d data
        data, emotions, actors = reshapeData(data, emotions, actors, frames)

        # filter by actor
        if actor is not None:
            data = data[actors == actor,:]
            emotions = emotions[actors == actor]
            actors = actors[actors == actor]
        
        # filter by emotion
        if emotion is not None:
            data = data[emotions == emotion,:]
            actors = actors[emotions == emotion]
            emotions = emotions[emotions == emotion]
            
    
    elif type=='3d':
        # filter by actor
        if actor is not None:
            data = data[actors == actor,:,:]
            emotions = emotions[actors == actor]
            actors = actors[actors == actor]
        
        # filter by emotion
        if emotion is not None:
            data = data[emotions == emotion,:,:]
            actors = actors[emotions == emotion]    
            emotions = emotions[emotions == emotion]
            
    
    return (data, emotions, actors)

In [26]:
# get all data in 3d
data, emotions, actors = getData(type='3d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get all data in 2d
data, emotions, actors = getData(type='2d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from actor 1 in 3d
data, emotions, actors = getData(actor=1, type='3d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from actor 1 in 2d
data, emotions, actors = getData(actor=1, type='2d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from emotion 1 in 3d
data, emotions, actors = getData(emotion=1, type='3d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from emotion 1 in 2d
data, emotions, actors = getData(emotion=1, type='2d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from actor 1 with emotion 1 in 3d
data, emotions, actors = getData(actor=1, emotion=1, type='3d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())

# get data from actor 1 with emotion 1 in 2d
data, emotions, actors = getData(actor=1, emotion=1, type='2d')
print(data.shape, emotions.shape, actors.shape, np.isnan(data).any())


(2452, 136, 191) (2452,) (2452,) True
(299854, 136) (299854,) (299854,) False
(104, 136, 191) (104,) (104,) True
(12668, 136) (12668,) (12668,) False
(188, 136, 191) (188,) (188,) True
(22458, 136) (22458,) (22458,) False
(8, 136, 191) (8,) (8,) True
(888, 136) (888,) (888,) False


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/01-01-01-01-01-01-01.csv')

In [9]:
df['timestamp'][30]

1.001

In [10]:
df

,frame,timestamp,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0.000,0.98,0.068407,0.337437,-0.938859,-0.130211,0.294125,-0.946856,-0.033,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,2,0.033,0.98,0.170199,0.301091,-0.938284,-0.077985,0.228398,-0.970440,0.048,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3,0.067,0.98,0.169057,0.236200,-0.956885,-0.106635,0.242925,-0.964166,0.032,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.100,0.98,0.145077,0.238042,-0.960359,-0.109159,0.177592,-0.978031,0.019,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.133,0.98,0.155419,0.221055,-0.962798,-0.084080,0.144999,-0.985853,0.037,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,3.103,0.98,0.023954,0.169759,-0.985194,-0.085197,0.218760,-0.972052,-0.031,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
94,95,3.136,0.98,0.027718,0.194906,-0.980430,-0.091073,0.220140,-0.971207,-0.032,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95,96,3.170,0.98,0.015107,0.174879,-0.984474,-0.091575,0.221444,-0.970864,-0.039,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
96,97,3.203,0.98,0.014757,0.178140,-0.983895,-0.090219,0.226799,-0.969754,-0.039,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
